In [1]:
# libraries
import json
import pandas as pd

**What's in the Data?** 
Code attribution: Taken from the MS COCO API Code (next cell only)

In [13]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
dataDir=''
dataType='val2017'
annFile='annotations/instances_{}.json'.format(dataType)
# initialize COCO api for instance annotations
coco=COCO(annFile)
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('\n')
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

loading annotations into memory...
Done (t=0.28s)
creating index...
index created!


COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush

COCO supercategories: 
person indoor sports appliance outdoor furniture accessory electronic food kitchen vehicle animal


**Data Definitions**

In [2]:
# paths

## captions
train_annot = 'annotations/captions_train2017.json'
val_annot = 'annotations/captions_val2017.json'

## instances
train_inst = 'annotations/instances_train2017.json'
val_inst = 'annotations/instances_val2017.json'

In [3]:
def load_json(path):
    f = open(path)
    f = json.load(f)
    return f

In [4]:
cap_val_data = load_json(val_annot) # captions
inst_val_data = load_json(val_inst)

In [5]:
annot_cap = cap_val_data['annotations']  # has the captions
images_cap = cap_val_data['images']  # map file name to captions data

cat = inst_val_data['categories']   # map cat id to cat
images_inst = inst_val_data['images']  # map filename to image id
annot_inst = inst_val_data['annotations'] # map image id to cat id

In [6]:
# dataframes from captions data
df_images_cap = pd.DataFrame(images_cap)
df_annot_cap = pd.DataFrame(annot_cap)

# dataframes from category information
df_images_inst = pd.DataFrame(images_inst)
df_cat = pd.DataFrame(cat)
df_annot_inst = pd.DataFrame(annot_inst)
df_cat[df_cat.supercategory == 'outdoor']

,supercategory,id,name
9,outdoor,10,traffic light
10,outdoor,11,fire hydrant
11,outdoor,13,stop sign
12,outdoor,14,parking meter
13,outdoor,15,bench


**Instance File**

In [7]:
df_images_inst

,license,file_name,coco_url,height,width,date_captured,flickr_url,id
0,4,000000397133.jpg,http://images.cocodataset.org/val2017/00000039...,427,640,2013-11-14 17:02:52,http://farm7.staticflickr.com/6116/6255196340_...,397133
1,1,000000037777.jpg,http://images.cocodataset.org/val2017/00000003...,230,352,2013-11-14 20:55:31,http://farm9.staticflickr.com/8429/7839199426_...,37777
2,4,000000252219.jpg,http://images.cocodataset.org/val2017/00000025...,428,640,2013-11-14 22:32:02,http://farm4.staticflickr.com/3446/3232237447_...,252219
3,1,000000087038.jpg,http://images.cocodataset.org/val2017/00000008...,480,640,2013-11-14 23:11:37,http://farm8.staticflickr.com/7355/8825114508_...,87038
4,6,000000174482.jpg,http://images.cocodataset.org/val2017/00000017...,388,640,2013-11-14 23:16:55,http://farm8.staticflickr.com/7020/6478877255_...,174482
...,...,...,...,...,...,...,...,...
4995,3,000000512403.jpg,http://images.cocodataset.org/val2017/00000051...,640,529,2013-11-24 05:12:53,http://farm1.staticflickr.com/143/350452845_fa...,512403
4996,4,000000168974.jpg,http://images.cocodataset.org/val2017/00000016...,500,375,2013-11-24 07:19:48,http://farm3.staticflickr.com/2360/2063838083_...,168974
4997,1,000000552775.jpg,http://images.cocodataset.org/val2017/00000055...,500,375,2013-11-24 10:38:31,http://farm4.staticflickr.com/3136/3106037881_...,552775
4998,3,000000394940.jpg,http://images.cocodataset.org/val2017/00000039...,640,426,2013-11-24 13:47:05,http://farm9.staticflickr.com/8227/8566023505_...,394940


In [8]:
df_combined_cat_info = df_annot_inst
df_combined_cat_info = df_combined_cat_info.merge(df_cat, how='left', left_on='category_id', right_on='id',suffixes=('', '_cat'))[['image_id','category_id', 'id', 'supercategory','name']]
df_combined_cat_info

,image_id,category_id,id,supercategory,name
0,289343,18,1768,animal,dog
1,61471,18,1773,animal,dog
2,472375,18,2551,animal,dog
3,520301,18,3186,animal,dog
4,579321,18,3419,animal,dog
...,...,...,...,...,...
36776,15517,6,900600015517,vehicle,bus
36777,439994,1,900100439994,person,person
36778,117719,44,904400117719,kitchen,bottle
36779,50149,52,905200050149,food,banana


In [9]:
# deficit cell - should be equal
df_combined_cat_info.image_id.nunique() # 4952
df_images_inst.id.nunique() # 5000

set(df_images_inst.id.to_list()) == set(df_combined_cat_info.image_id.to_list())

False

In [10]:
df_cat_final = df_combined_cat_info
df_cat_final = df_cat_final.merge(df_images_inst, how='inner', left_on='image_id', right_on='id', suffixes=('', '_cat'))[['image_id','category_id','id','supercategory','name','file_name','coco_url']]
df_cat_final

,image_id,category_id,id,supercategory,name,file_name,coco_url
0,289343,18,1768,animal,dog,000000289343.jpg,http://images.cocodataset.org/val2017/00000028...
1,289343,1,201376,person,person,000000289343.jpg,http://images.cocodataset.org/val2017/00000028...
2,289343,15,575188,outdoor,bench,000000289343.jpg,http://images.cocodataset.org/val2017/00000028...
3,289343,2,2168354,vehicle,bicycle,000000289343.jpg,http://images.cocodataset.org/val2017/00000028...
4,61471,18,1773,animal,dog,000000061471.jpg,http://images.cocodataset.org/val2017/00000006...
...,...,...,...,...,...,...,...
36776,429530,86,1994143,indoor,vase,000000429530.jpg,http://images.cocodataset.org/val2017/00000042...
36777,429530,86,1994241,indoor,vase,000000429530.jpg,http://images.cocodataset.org/val2017/00000042...
36778,429530,86,1994424,indoor,vase,000000429530.jpg,http://images.cocodataset.org/val2017/00000042...
36779,31749,85,2197091,indoor,clock,000000031749.jpg,http://images.cocodataset.org/val2017/00000003...


In [11]:
sum(df_cat_final.supercategory == 'animal')

# sum(df_cat_final.name == 'person')
# person = df_cat_final[df_cat_final.name == 'person'].reset_index()
# person['coco_url'][10]

2700

*Decide which "supercategory" or "name" you want here; the data you need from this half is in data_cat_final*

In [ ]:
partition = df_cat_final[df_cat_final.supercategory == 'animal'].reset_index()
partition

,index,image_id,category_id,id,supercategory,name,file_name,coco_url
0,0,289343,18,1768,animal,dog,000000289343.jpg,http://images.cocodataset.org/val2017/00000028...
1,4,61471,18,1773,animal,dog,000000061471.jpg,http://images.cocodataset.org/val2017/00000006...
2,7,472375,18,2551,animal,dog,000000472375.jpg,http://images.cocodataset.org/val2017/00000047...
3,11,520301,18,3186,animal,dog,000000520301.jpg,http://images.cocodataset.org/val2017/00000052...
4,12,579321,18,3419,animal,dog,000000579321.jpg,http://images.cocodataset.org/val2017/00000057...
...,...,...,...,...,...,...,...,...
2695,34905,120420,16,2066566,animal,bird,000000120420.jpg,http://images.cocodataset.org/val2017/00000012...
2696,34906,120420,16,2066584,animal,bird,000000120420.jpg,http://images.cocodataset.org/val2017/00000012...
2697,35075,159791,16,1400234,animal,bird,000000159791.jpg,http://images.cocodataset.org/val2017/00000015...
2698,35178,243344,17,1816125,animal,cat,000000243344.jpg,http://images.cocodataset.org/val2017/00000024...


**Captions File**

In [ ]:
df_annot_cap

,image_id,id,caption
0,179765,38,A black Honda motorcycle parked in front of a ...
1,179765,182,A Honda motorcycle parked in a grass driveway
2,190236,401,An office cubicle with four different types of...
3,331352,441,A small closed toilet in a cramped space.
4,517069,447,Two women waiting at a bench next to a street.
...,...,...,...
25009,9590,821635,A group of men sipping drinks and talking at a...
25010,84664,822557,"A plate of food with some eggs, potatoes, brea..."
25011,331569,824852,The strawberries was sitting beside the tall g...
25012,231237,825902,A bunch of small red flowers in a barnacle enc...


In [ ]:
df_cap_final = df_annot_cap.groupby('image_id')['caption'].apply(list).reset_index()
df_cap_final

,image_id,caption
0,139,[A woman stands in the dining area at the tabl...
1,285,[A big burly grizzly bear is show with grass i...
2,632,"[Bedroom scene with a bookcase, blue comforter..."
3,724,[A stop sign is mounted upside-down on it's po...
4,776,"[Three teddy bears, each a different color, sn..."
...,...,...
4995,581317,"[A woman holding a small item in a field., Wom..."
4996,581357,[Skate boarder hang time with young man appear...
4997,581482,[A fancy metal cloth is seen against huge barr...
4998,581615,[A picture of a white toilet in the rest room....


In [ ]:
set(df_images_cap.id.to_list()) == set(df_annot_cap.image_id.to_list())

True

**Combining Both**

In [ ]:
df_final = partition
df_final = df_final.merge(df_cap_final, how='inner', left_on='image_id', right_on='image_id', suffixes=('', '_cat'))
df_final.loc[:,df_final.columns!='index']

,image_id,category_id,id,supercategory,name,file_name,coco_url,caption
0,289343,18,1768,animal,dog,000000289343.jpg,http://images.cocodataset.org/val2017/00000028...,[A man is riding a large bike through the park...
1,61471,18,1773,animal,dog,000000061471.jpg,http://images.cocodataset.org/val2017/00000006...,[Puppy chewing on toilet paper in the bathroom...
2,472375,18,2551,animal,dog,000000472375.jpg,http://images.cocodataset.org/val2017/00000047...,[A small black and brown dog sitting next to a...
3,520301,18,3186,animal,dog,000000520301.jpg,http://images.cocodataset.org/val2017/00000052...,[A brown dog hanging it's head out of a car wi...
4,579321,18,3419,animal,dog,000000579321.jpg,http://images.cocodataset.org/val2017/00000057...,[The person walking the dog has dressed it in ...
...,...,...,...,...,...,...,...,...
2695,120420,16,2066566,animal,bird,000000120420.jpg,http://images.cocodataset.org/val2017/00000012...,[A man holding a silver flip phone open in fro...
2696,120420,16,2066584,animal,bird,000000120420.jpg,http://images.cocodataset.org/val2017/00000012...,[A man holding a silver flip phone open in fro...
2697,159791,16,1400234,animal,bird,000000159791.jpg,http://images.cocodataset.org/val2017/00000015...,[A white refrigerator freezer covered in magne...
2698,243344,17,1816125,animal,cat,000000243344.jpg,http://images.cocodataset.org/val2017/00000024...,[Awake cat near ceiling on a closed refrigerat...


In [ ]:
df_final.to_csv('animals.csv')